In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd 

from tqdm import tqdm
from utils import *

In [ ]:
train_set, train_labels, test_set, test_labels = load_and_preprocess_data(path='./data/CIFAR10')

In [ ]:
mu = 1e-3
rho = 1e-4

input_size = 32 * 32 * 3
hidden_size = 2048
output_size = 10

In [ ]:
def train_network(model: NeuralNetwork, train_set, train_labels, num_epochs, p_dropout=None):
    N_samples = train_set.shape[0]

    # Add accuracy and loss tracking

    for epoch in range(num_epochs):
        
        data_idx = np.random.permutation(N_samples)
        running_epoch_loss = 0
        running_predictions = []
        running_labels = []
        
        pbar = tqdm(total=N_samples)
        for iter, idx in enumerate(data_idx):
            h = train_set[idx]
            gamma = transform_label(train_labels[idx])

            gamma_hat = model.forward(h)
            loss = cross_entropy_loss(gamma_hat, gamma, model.rho, model.weights())
            model.backward(gamma_hat, gamma)

            running_epoch_loss += loss
    
            running_predictions.append(np.argmax(gamma_hat))
            running_labels.append(train_labels[idx])

            pbar.update(1)
            pbar.set_description(f'Train loss: {loss:.4f}')
        pbar.close()

        epoch_loss = running_epoch_loss / N_samples
        epoch_accuracy = np.mean(np.array(running_predictions) == np.array(running_labels))

        print(f"Epoch: {epoch + 1} | Train Loss {epoch_loss:.4f} | Train Accuracy: {epoch_accuracy * 100:.2f}%")

    model.normalize_weights()

In [ ]:
def evaluate(model: NeuralNetwork, dataset, labels):
    N_samples = dataset.shape[0]

    running_predictions = []
    running_labels = []

    pbar = tqdm(total=N_samples)
    for idx in range(N_samples):
        h = dataset[idx]
        gamma = transform_label(labels[idx])
        gamma_hat = model.forward(h)

        running_predictions.append(np.argmax(gamma_hat))
        running_labels.append(labels[idx])

        pbar.update(1)
        pbar.set_description("Evaluating...")
    pbar.close()

    num_misclassified = np.sum(np.array(running_predictions) != np.array(running_labels))
    error = num_misclassified / N_samples
    accuracy = 1 - error

    print(f"Accuracy: {accuracy * 100:.2f}% | Error: {error * 100:.2f}% | Misclassified: {num_misclassified}")

    return accuracy, error, num_misclassified


### Training without dropout

In [ ]:
model = NeuralNetwork(input_size, hidden_size, output_size, ReLU, d_ReLU, mu, rho)

In [ ]:
train_network(model, train_set[:5000], train_labels[:5000], num_epochs=10)

In [ ]:
train_acc, train_err, train_misc = evaluate(model, train_set[:5000], train_labels[:5000])

In [ ]:
test_acc, test_err, test_misc = evaluate(model, test_set, test_labels)

### Training with dropout

In [ ]:
model = NeuralNetwork(input_size, hidden_size, output_size, ReLU, d_ReLU, mu, rho)

In [ ]:
train_network(model, train_set[:5000], train_labels[:5000], num_epochs=15, p_dropout=[0.1, 0.5, 0.5])

In [ ]:
train_acc_do, train_err_do, train_misc_do = evaluate(model, train_set, train_labels)

In [ ]:
test_acc_do, test_err_do, test_misc_do = evaluate(model, test_set, test_labels)

### Tables

In [ ]:
pd.DataFrame(
    {
        "Setting": "w/o Dropout",
        "Test Error [%]": test_err,
        "# Test Misclassified": test_misc,
        "Train Error [%]": train_err,
        "# Train Misclassified": train_misc,
    },
    {
        "Setting": "w/ Dropout",
        "Test Error [%]": test_err_do,
        "# Test Misclassified": test_misc_do,
        "Train Error [%]": train_err_do,
        "# Train Misclassified": train_misc_do,
    }
)